In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold




In [ ]:
def run_rating_models_cross_validation (fa_cup_scores_df, ratings_function, ratings_model, num_folds):

    # Create a StratifiedKFold object and fold counter
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_counter = 0

    # Initialize empty lists to store results
    predictions = []
    fold_ratings = []
    accuracies = []
    all_actual_upsets = []
    all_predicted_upsets = []

    # Iterate over the folds
    for fold, (train_idx, test_idx) in enumerate(skf.split(fa_cup_scores_df, fa_cup_scores_df['actual_upset']), start=1):
        print(f"Fold {fold}/{num_folds}")
        fold_counter += 1  # Increment fold_counter

        # Split the data into training and test sets
        train_data = fa_cup_scores_df.iloc[train_idx]
        test_data = fa_cup_scores_df.iloc[test_idx]
        print(f"Train data size: {len(train_data)}")
        print(f"Test data size: {len(test_data)}")

        # Get distinct teams
        all_teams = pd.concat([train_data['home_team_no'], train_data['away_team_no']]).drop_duplicates().sort_values().reset_index(drop=True)
        print(ratings_model)
        print(ratings_function)
        # Call rating function
        if ratings_model == 'borda_count':
            ratings = ratings_function(fold_counter)
        elif ratings_model == 'average_rank':
            ratings = ratings_function(fold_counter)
        elif ratings_model == 'local_kemeny_optimisation':
            ratings = ratings_function(fold_counter)
        else:
            ratings = ratings_function(train_data)
        print(ratings)
        # Append the ratings to the list
        ratings_list = [(ratings_model, fold, team_no, rating) for team_no, rating in ratings]
        fold_ratings.extend(ratings_list)

        # Predict upsets and calculate accuracy
        actual_upsets = []
        predicted_upsets = []

        for _, row in test_data.iterrows():
            predicted_winner, predicted_upset, home_team_rating, away_team_rating = predict_winner_upset(
                row['home_team_no'], row['away_team_no'],
                row['home_team_league_level'], row['away_team_league_level'],
                ratings, home_advantage=0
            )
            actual_upset = row['actual_upset']
            actual_upsets.append(actual_upset)
            predicted_upsets.append(predicted_upset)
            predictions.append({
                'ratings_model': ratings_model,
                'fold_number': fold,
                'match_id': row['match_id'],
                'home_team_no': row['home_team_no'],
                'home_team_league_level': row['home_team_league_level'],
                'away_team_no': row['away_team_no'],
                'away_team_league_level': row['away_team_league_level'],
                'home_team_rating': home_team_rating,
                'away_team_rating': away_team_rating,
                'predicted_winner': predicted_winner,
                'actual_winner': row['actual_winning_team_no'],
                'actual_upset': actual_upset,
                'predicted_upset': predicted_upset,
            })

        # Calculate accuracy
        upset_accuracy = accuracy_score(actual_upsets, predicted_upsets)
        print(f"Accuracy score: {upset_accuracy}")
        accuracies.append(upset_accuracy)

        # Store true and predicted values for later analysis
        all_actual_upsets.extend(actual_upsets)
        all_predicted_upsets.extend(predicted_upsets)

    # Create DataFrames from results
    predictions_df = pd.DataFrame(predictions)
    predictions_df.index = range(1, len(predictions) + 1)

    upset_accuracy_scores_df = pd.DataFrame(accuracies, columns=['accuracy'])
    upset_accuracy_scores_df.index = range(1, len(accuracies) + 1)

    # Fold ratings dataframe
    fold_ratings_df = pd.DataFrame(fold_ratings, columns=['ratings_model','fold_number','team_no', 'rating'])
    fold_ratings_df = fold_ratings_df.sort_values('team_no', ascending=True)
    #fold_ratings_df.index = range(1, len(fold_ratings) + 1)
    fold_ratings_df['rank'] = fold_ratings_df.groupby('fold_number')['rating'].rank(ascending=False, method='dense').astype(int)

    # Mean ratings from 5 folds. Assign 0 fold as mean
    mean_ratings_df = fold_ratings_df.groupby('team_no')['rating'].mean().reset_index()
    mean_ratings_by_team_df = pd.DataFrame({'ratings_model': ratings_model, 'fold_number': 0,'team_no': mean_ratings_df['team_no'],'rating': mean_ratings_df['rating']})
    mean_ratings_by_team_df = mean_ratings_by_team_df.sort_values('rating', ascending=False)
    #mean_ratings_by_team_df.index = range(1, len(mean_ratings_by_team_df) + 1)
    mean_ratings_by_team_df['rank'] = mean_ratings_by_team_df.groupby('fold_number')['rating'].rank(ascending=False, method='dense').astype(int)

    # Combine and sort DataFrame
    ratings_df = pd.concat([fold_ratings_df, mean_ratings_by_team_df], ignore_index=True)
    ratings_df = ratings_df.sort_values(['team_no', 'fold_number'], ascending=[True, True])

    # Calculate overall metrics
    mean_upset_accuracy = np.mean(accuracies)
    std_upset_accuracy = np.std(accuracies)

    return {
        'predictions_df': predictions_df,
        'upset_accuracy_scores_df': upset_accuracy_scores_df,
        'ratings_df': ratings_df,
        'mean_upset_accuracy': mean_upset_accuracy,
        'std_upset_accuracy': std_upset_accuracy,
        'accuracies': accuracies,
        'all_actual_upsets': all_actual_upsets,
        'all_predicted_upsets': all_predicted_upsets
    }


In [ ]:
def predict_winner_upset(home_team_no, away_team_no, home_team_league_level, away_team_league_level, ratings, home_advantage=0):
    home_team_rating = next((rating for team_no, rating in ratings if team_no == home_team_no), None) + home_advantage
    away_team_rating = next((rating for team_no, rating in ratings if team_no == away_team_no), None)

    if home_team_rating is None or away_team_rating is None:
        raise ValueError("Team number not found in ratings list.")

    if home_team_rating > away_team_rating:
        predicted_winner = home_team_no
        if home_team_league_level > away_team_league_level:
            predicted_upset = 1
        else:
            predicted_upset = 0
    else:
        predicted_winner = away_team_no
        if away_team_league_level > home_team_league_level:
            predicted_upset = 1
        else:
            predicted_upset = 0

    return predicted_winner, predicted_upset, home_team_rating, away_team_rating

In [1]:
def create_model_results_df(all_y_true, all_y_pred, accuracies, model_name):
    report_dict = classification_report(all_y_true, all_y_pred, output_dict=True)
    cm = confusion_matrix(all_y_true, all_y_pred)

    metrics = []
    values = []

    # Add cross-validation accuracy
    avg_accuracy = np.mean(accuracies)
    metrics.append('Cross-validation Accuracy')
    values.append(avg_accuracy)

    # Add overall accuracy from the classification report
    metrics.append('Overall Accuracy')
    values.append(report_dict['accuracy'])

    # Add confusion matrix values
    cm_labels = ['True Negative (Class 0)', 'False Positive (Class 1)', 'False Negative (Class 0)', 'True Positive (Class 1)']
    for label, value in zip(cm_labels, cm.ravel()):
        metrics.append(f'Confusion Matrix - {label}')
        values.append(value)

    # Add precision, recall, and f1-score for each class
    for class_label in sorted(report_dict.keys()):
        if class_label.isdigit():  # This checks if the key is a class label
            for metric in ['precision', 'recall', 'f1-score']:
                metrics.append(f'{metric.capitalize()} (Class {class_label})')
                values.append(report_dict[class_label][metric])

    # Add macro and weighted averages
    for avg_type in ['macro avg', 'weighted avg']:
        for metric in ['precision', 'recall', 'f1-score']:
            metrics.append(f'{avg_type.capitalize()} {metric.capitalize()}')
            values.append(report_dict[avg_type][metric])

    # Calculate and add AUC-ROC score
    auc_roc = roc_auc_score(all_y_true, all_y_pred)
    metrics.append('AUC-ROC')
    values.append(auc_roc)

    # Create the DataFrame
    results_df = pd.DataFrame({
        'metric': metrics,
        model_name: values
    })

    # Format the numeric values to 3 decimal places
    results_df[model_name] = results_df[model_name].apply(lambda x: f'{x:.3f}' if isinstance(x, (int, float)) else str(x))

    # Present the confusion matrix
    cm_fig, ax = plt.subplots(figsize=(10,7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax)
    ax.set_title(f'Confusion Matrix {model_name}')
    ax.set_ylabel('Actual Class\n(0 not upset and 1 upset)')
    ax.set_xlabel('Predicted Class\n(0 not upset and 1 upset)')

    # Plot ROC curve
    fpr, tpr, _ = roc_curve(all_y_true, all_y_pred)
    roc_fig, roc_ax = plt.subplots(figsize=(8,6))
    roc_ax.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_roc:.2f})')
    roc_ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    roc_ax.set_xlim([0.0, 1.0])
    roc_ax.set_ylim([0.0, 1.05])
    roc_ax.set_xlabel('False Positive Rate')
    roc_ax.set_ylabel('True Positive Rate')
    roc_ax.set_title(f'Receiver Operating Characteristic (ROC) Curve - {model_name}')
    roc_ax.legend(loc="lower right")

    return results_df, cm_fig, roc_fi

    return results_df